# W5-CRISPR-Cas9-inframe_deletion

In Workflow 5, CRISPR-Cas9 combines ssDNA bridging and Gibson cloning to create in-frame deletions (Figure 6A). This ensures that the downstream genes are not disrupted, which is crucial for functional studies of gene knockouts. This workflow can be used to study gene function by knocking out specific genes in streptomycetes and observing the resulting phenotypic changes. It can also be used to create markerless deletions, which are important for constructing clean genetic backgrounds for further genetic manipulations. To get started, download the pCRISPR-Cas9 file and a genome file (e.g., S. coelicolor A3), then select the genes to delete. StreptoAIM will generate the necessary primers and plasmid constructs (S5).


In [1]:
import sys
import os
from Bio.Restriction import StuI
from pydna.dseqrecord import Dseqrecord
import pandas as pd
from datetime import datetime


# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from streptocad.sequence_loading.sequence_loading import (
    load_and_process_gene_sequences,
    load_and_process_genome_sequences,
    load_and_process_plasmid, 
    check_and_convert_input,
    annotate_dseqrecord,
    process_specified_gene_sequences_from_record)

from streptocad.utils import polymerase_dict, create_primer_df_from_dict,ProjectDirectory,extract_metadata_to_dataframe
from streptocad.primers.primer_generation import create_idt_order_dataframe
from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging,make_ssDNA_oligos
from streptocad.crispr.guideRNAcas3_9_12 import extract_sgRNAs, SgRNAargs
from streptocad.cloning.gibson_cloning import (
    find_up_dw_repair_templates,
    assemble_multiple_plasmids_with_repair_templates_for_deletion,
    update_primer_names
)

from streptocad.cloning.plasmid_processing import check_plasmid_restriction_sites, determine_workflow_order_for_plasmids
from streptocad.primers.primer_generation import checking_primers, primers_to_IDT, find_best_check_primers_from_genome

In [2]:
from pydna.dseqrecord import Dseqrecord

def find_locus_tags(record: Dseqrecord, max_tags: int = None):
    """
    Iterate through a Dseqrecord and return a list of unique locus tags found in its features,
    stopping after `max_tags` tags if specified.

    :param record: pydna.dseqrecord.Dseqrecord object
    :param max_tags: maximum number of locus tags to collect (None for no limit)
    :return: list of locus_tag strings
    """
    tags = []
    seen = set()
    count = 0

    # Ensure the record has features
    if not hasattr(record, 'features'):
        return tags

    for feature in record.features:
        if max_tags is not None and count >= max_tags:
            break

        qualifiers = getattr(feature, 'qualifiers', {})

        for tag in qualifiers.get('locus_tag', []):
            if max_tags is not None and count >= max_tags:
                break
            # only append new tags
            if tag not in seen:
                seen.add(tag)
                tags.append(tag)
                count += 1

        if max_tags is not None and count >= max_tags:
            break

    return tags


## INPUT

In [3]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../../../data/plasmids/pCRISPR–Cas9_plasmid_addgene.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
locus_tags = find_locus_tags(genome, 10)
genes_to_KO = locus_tags

#### Advanced settings ####
# 4 Filtering metrics for sgRNAs
gc_upper = 0.72
gc_lower = 0.2
off_target_seed = 13
off_target_upper = 10
cas_type='cas9'
number_of_sgRNAs_per_group = 1

# 5 Choose polymerase and target melting temperature
chosen_polymerase = polymerase_dict['Phusion High-Fidelity DNA Polymerase (GC Buffer)']
melting_temperature = 65
primer_concentration = 0.4 
primer_number_increment = 23
flanking_region = 500

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('CGGTTGGTAGGATCGACGGC')
dw_homology = Dseqrecord('GTTTTAGAGCTAGAAATAGC')

# 7 extra settings
#### Advanced settings ####
repair_length = 1000
overlap_length = 40 

# Computation

In [4]:
target_dict, genes_to_KO, annotation_input = check_and_convert_input(genes_to_KO)

print(annotation_input)
if annotation_input == True:
    genome = annotate_dseqrecord(genome, target_dict)

False


In [5]:
len(genes_to_KO)

10

In [6]:
# Initialize SgRNAargs with desired parameters
args = SgRNAargs(genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9'
                )

sgrna_df = extract_sgRNAs(args)
sgrna_df

sgRNA generated were outside the designated border in SCO0001. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0001. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO0002. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRN

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
0,NC_003888.3,SCO0001,446,1,-1,7,0.55,CGG,TCCGGTAGATTCGTGGTGAC,GATTCGTGGTGAC,1
1590,NC_003888.3,SCO0005,7187,1,-1,1724,0.55,TGG,TGGCTTCATTGAAGGGCTGG,ATTGAAGGGCTGG,1
1587,NC_003888.3,SCO0005,7187,1,-1,1701,0.60,CGG,ACGTGGTCGCGGTGAATCCA,CGCGGTGAATCCA,1
1586,NC_003888.3,SCO0005,7187,1,-1,1700,0.65,GGG,CGTGGTCGCGGTGAATCCAC,GCGGTGAATCCAC,1
1585,NC_003888.3,SCO0005,7187,1,-1,1678,0.50,CGG,GATTTCGTGAAGCTGTCCGT,TGAAGCTGTCCGT,1
...,...,...,...,...,...,...,...,...,...,...,...
342,NC_003888.3,SCO0002,1252,1,-1,1672,0.65,CGG,ATAGGGCACGGCGTACAGGC,ACGGCGTACAGGC,5
1609,NC_003888.3,SCO0005,7187,1,-1,1847,0.70,TGG,TCTGCTCGGTCTCGGCGAGG,GGTCTCGGCGAGG,5
633,NC_003888.3,SCO0002,1252,1,1,566,0.65,TGG,GGAGCTGGTCATCGCGTCGA,GTCATCGCGTCGA,5
325,NC_003888.3,SCO0002,1252,1,-1,1564,0.70,CGG,TGTGAGGGCGGCGATGTCGG,GCGGCGATGTCGG,5


In [7]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
0,NC_003888.3,SCO0001,446,1,-1,7,0.55,CGG,TCCGGTAGATTCGTGGTGAC,GATTCGTGGTGAC,1
1590,NC_003888.3,SCO0005,7187,1,-1,1724,0.55,TGG,TGGCTTCATTGAAGGGCTGG,ATTGAAGGGCTGG,1
1206,NC_003888.3,SCO0004,6226,1,-1,374,0.70,CGG,GGCCGGTCACGTCCGGATTC,CACGTCCGGATTC,1
1183,NC_003888.3,SCO0003,3869,1,1,1,0.60,TGG,TGACGGATCCGTTTTCCCCG,TCCGTTTTCCCCG,1
2598,NC_003888.3,SCO0010,12366,1,1,6,0.70,GGG,GGCCGCACCCTGCGATTCCT,CCCTGCGATTCCT,1
2363,NC_003888.3,SCO0009,11617,-1,-1,35,0.70,TGG,CGAGTGCTGCCAGCTGCTCC,TGCCAGCTGCTCC,1
2303,NC_003888.3,SCO0008,11010,-1,-1,10,0.60,TGG,TGGGTGAGGAAGACGCTGAG,GGAAGACGCTGAG,1
1992,NC_003888.3,SCO0006,9304,1,-1,776,0.65,CGG,CGCAGTCGATGAGGGAGGCA,GATGAGGGAGGCA,1
2204,NC_003888.3,SCO0007,10554,-1,-1,71,0.70,GGG,GGGCTGCTTGCTGGTCGCGT,TTGCTGGTCGCGT,1
474,NC_003888.3,SCO0002,1252,1,1,2145,0.70,GGG,GACCCCGGGGTGCTGTTCGA,GGGTGCTGTTCGA,1


## Output

In [8]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)
print(list_of_ssDNAs[0].name)

# cut plasmid
from Bio.Restriction import NcoI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI), key=lambda x: len(x), reverse=True)[0]
#print(linearized_plasmid)

sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)
len(sgRNA_vectors)

SCO0001_loc_7


[Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279)]

In [9]:
# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"pCas9_{row['locus_tag']}({row['sgrna_loc']})"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'{targeting_info[i]}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  
    sgRNA_vectors[i].description = f'CRISPR-Cas9 targeting {", ".join(genes_to_KO)} for single gene knockout, assembled using StreptoCAD.'

In [10]:
print_plasmids = False

if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR-Cas9/{vector.id}.gb")

### IDT primers

In [11]:
idt_primers=primers_to_IDT(list_of_ssDNAs)
idt_df = idt_primers
idt_df

,Name,Sequence,Concentration,Purification
0,SCO0001_loc_7,CGGTTGGTAGGATCGACGGCTCCGGTAGATTCGTGGTGACGTTTTA...,25nm,STD
1,SCO0005_loc_1724,CGGTTGGTAGGATCGACGGCTGGCTTCATTGAAGGGCTGGGTTTTA...,25nm,STD
2,SCO0004_loc_374,CGGTTGGTAGGATCGACGGCGGCCGGTCACGTCCGGATTCGTTTTA...,25nm,STD
3,SCO0003_loc_1,CGGTTGGTAGGATCGACGGCTGACGGATCCGTTTTCCCCGGTTTTA...,25nm,STD
4,SCO0010_loc_6,CGGTTGGTAGGATCGACGGCGGCCGCACCCTGCGATTCCTGTTTTA...,25nm,STD
5,SCO0009_loc_35,CGGTTGGTAGGATCGACGGCCGAGTGCTGCCAGCTGCTCCGTTTTA...,25nm,STD
6,SCO0008_loc_10,CGGTTGGTAGGATCGACGGCTGGGTGAGGAAGACGCTGAGGTTTTA...,25nm,STD
7,SCO0006_loc_776,CGGTTGGTAGGATCGACGGCCGCAGTCGATGAGGGAGGCAGTTTTA...,25nm,STD
8,SCO0007_loc_71,CGGTTGGTAGGATCGACGGCGGGCTGCTTGCTGGTCGCGTGTTTTA...,25nm,STD
9,SCO0002_loc_2145,CGGTTGGTAGGATCGACGGCGACCCCGGGGTGCTGTTCGAGTTTTA...,25nm,STD


# In frame deletion

In [12]:
in_frame_deletion = False


In [13]:
if in_frame_deletion:
    

    # Make repair templates
    repair_templates_data = find_up_dw_repair_templates(genome, 
                                                        genes_to_KO, 
                                                        target_tm=melting_temperature, 
                                                        primer_tm_kwargs={'conc':primer_concentration, 'prodcode':chosen_polymerase} , 
                                                        repair_length=repair_length, )

    # Digest the plasmids
    processed_records = [Dseqrecord(record, circular=True).cut(StuI)[0] for record in sgRNA_vectors]

    # Rename them appropriately
    for i in range(len(processed_records)):
        processed_records[i].name = sgRNA_vectors[i].name
        print(processed_records[i].name)

    # Assembly 
    assembly_data = assemble_multiple_plasmids_with_repair_templates_for_deletion(genes_to_KO, processed_records, 
                                                                                repair_templates_data, 
                                                                                overlap=overlap_length)
    # updating the primer names to something systematic.
    update_primer_names(assembly_data)
    print(assembly_data)

    # Parse through the primer df
    primer_df = create_primer_df_from_dict(assembly_data)
    unique_df = primer_df.drop_duplicates(keep='first')

    # IDT df
    idt_df = create_idt_order_dataframe(unique_df, concentration="25nm", purification="STD")

    # Contigs
    assembled_contigs = []
    for data in assembly_data: 
        contig_record = data['contig']
        contig_record.id = f"{data['name']}_w_rep"
        contig_record.name = f"{data['name']}_w_rep"
        assembled_contigs.append(contig_record)


In [14]:
if in_frame_deletion:
    print(unique_df)
    workflow_df = determine_workflow_order_for_plasmids(sgRNA_vectors, 
                                                            assembled_contigs,
                                                            ["StuI"], [ "NcoI"])
    print(workflow_df )
else: 
    workflow_df = pd.DataFrame()

In [15]:
if in_frame_deletion:
    integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
    plasmid_metadata_df = extract_metadata_to_dataframe(assembled_contigs,
                                                        clean_plasmid,
                                                        integration_names)
    workflow_df = determine_workflow_order_for_plasmids(sgRNA_vectors, 
                                                            assembled_contigs,
                                                            ["StuI",], [ "NcoI"])
    
    plasmid_metadata_df = pd.merge(plasmid_metadata_df, workflow_df, on='plasmid_name', how='inner')  # or 'left'/'right' depending on what you need

else: 
    integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
    plasmid_metadata_df = extract_metadata_to_dataframe(sgRNA_vectors,
                                                        clean_plasmid,
                                                        integration_names)
    workflow_df = pd.DataFrame()

plasmid_metadata_df

,plasmid_name,date,original_plasmid,integration,size
0,pCas9_SCO0001(7),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0001(7),11279
1,pCas9_SCO0005(1724),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0005(1724),11279
2,pCas9_SCO0004(374),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0004(374),11279
3,pCas9_SCO0003(1),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0003(1),11279
4,pCas9_SCO0010(6),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0010(6),11279
5,pCas9_SCO0009(35),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0009(35),11279
6,pCas9_SCO0008(10),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0008(10),11279
7,pCas9_SCO0006(776),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0006(776),11279
8,pCas9_SCO0007(71),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0007(71),11279
9,pCas9_SCO0002(2145),2025-06-10,pCRISPR-Cas9,sgRNA_SCO0002(2145),11279


## Folder with all the generated I/O

In [16]:
input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]

if in_frame_deletion: 
    output_files = [
        {"name": "cBEST_w_sgRNAs.gb", "content": assembled_contigs}, # LIST OF Dseqrecords
        {"name": "primer_df.csv", "content": primer_df},
        {"name": "sgrna_df.csv", "content": sgrna_df},
        {"name": "filtered_df.csv", "content": filtered_df},
        {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},

    ]
else: 
    output_files = [
        {"name": "cBEST_w_sgRNAs.gb", "content": sgRNA_vectors}, # LIST OF Dseqrecords
        {"name": "sgrna_df.csv", "content": sgrna_df},
        {"name": "filtered_df.csv", "content": filtered_df},
        {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},
        {"name": "workflow_order_df.csv", "content": workflow_df},

        workflow_df

    ]

input_values = {
    "genes_to_knockout": genes_to_KO,


    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group,

    },  
    "polymerase_settings": {
        "chosen_polymerase": chosen_polymerase,
        "melting_temperature": melting_temperature,
        "primer_concentration": primer_concentration,
        "primer_number_increment": primer_number_increment,
        "flanking_region": flanking_region
    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}


# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]


timestamp = datetime.utcnow().isoformat()

# Create project directory structure
project_directory = ProjectDirectory(
    project_name=f"CRISPR_cas9_plasmid_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths
)

# DO You want to save the folder? 
save_zip_folder = False 

if save_zip_folder: 
    # Generate the project directory structure and get the zip content
    zip_content = project_directory.create_directory_structure(create_directories=False)

    # Save the zip file to disk (optional)
    with open("project_structure.zip", "wb") as f:
        f.write(zip_content)